In [1]:
import numpy as np
import pandas as pd
import datetime

%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import style
style.use('ggplot')  # 加载'ggplot'风格
plt.rcParams['font.sans-serif'] = ['SimHei']  # 设置中文字体
plt.rcParams['axes.unicode_minus'] = False  # 正常显示负号

import sqlite3
import csv

from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf # 回归
from linearmodels.panel import PanelOLS # 面板回归
from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.tsa.stattools import adfuller # ADF
from statsmodels.regression.rolling import RollingOLS

from numpy import cumsum, log, polyfit, sqrt, std, subtract
from numpy.random import randn

from joblib import Parallel,delayed
from tqdm import tqdm,tnrange,tqdm_notebook   
import warnings
warnings.filterwarnings("ignore")

from TimeSeriesAnalysis import Univariate_Time_Series_Analysis, Multivariate_Time_Series_Analysis

# 数据载入、清洗、查看

In [2]:
def clean_mainmonth_info(future_i, future_code):
    month_num = future_i.shape[1]/4
    month_num = int(month_num) if month_num == int(month_num) else month_num

    future_i_mainmonth_info = pd.DataFrame()
    for j in range(month_num):
        future_i_month_j = future_i.iloc[3:,4*(j):4*(j+1)].dropna(how='all')
        future_i_month_j_bid_info = future_i_month_j.iloc[:,:2].dropna(how='all')
        future_i_month_j_ask_info = future_i_month_j.iloc[:,2:].dropna(how='all')
        future_i_month_j_bid_info = future_i_month_j_bid_info.set_index(future_i_month_j_bid_info.columns[0])
        future_i_month_j_ask_info = future_i_month_j_ask_info.set_index(future_i_month_j_ask_info.columns[0])

        future_i_month_j_info = pd.merge(future_i_month_j_bid_info, future_i_month_j_ask_info, left_index=True, right_index=True, how='outer')
        future_i_month_j_info.columns = [future_code+'_Bid', future_code+'_Ask']
        # 本合约交割前一时刻
        mainmonth_last_t = future_i_month_j_info.index.max()
        future_i_mainmonth_info = future_i_mainmonth_info[future_i_mainmonth_info.index > mainmonth_last_t]
        future_i_mainmonth_info = pd.concat([future_i_month_j_info, future_i_mainmonth_info])
        
    return future_i_mainmonth_info

In [3]:
# 载入汇率信息
USDCNY = pd.read_excel('./pairtrading/USDCNY historical 1min.xlsx').dropna(how='all').T.dropna(how='all').T.iloc[3:]

USDCNY_bid_info = USDCNY.iloc[:,:2].dropna(how='all')
USDCNY_ask_info = USDCNY.iloc[:,2:].dropna(how='all')
USDCNY_bid_info = USDCNY_bid_info.set_index(USDCNY_bid_info.columns[0])
USDCNY_ask_info = USDCNY_ask_info.set_index(USDCNY_ask_info.columns[0])

# 合并 排序 填充空值
USDCNY_info = pd.merge(USDCNY_bid_info, USDCNY_ask_info, left_index=True, right_index=True, how='outer')
USDCNY_info.sort_index(inplace=True)

USDCNY_info.columns = ['USDCNY_Bid', 'USDCNY_Ask']

In [4]:
# 载入数据，丢弃空行空列
future_CN = pd.read_excel('./pairtrading/CN.xlsx').dropna(how='all').T.dropna(how='all').T
future_IH = pd.read_excel('./pairtrading/IH.xlsx').dropna(how='all').T.dropna(how='all').T

# 合成主力合约价格信息
future_CN_mainmonth_info = clean_mainmonth_info(future_CN, 'CN').sort_index()
future_IH_mainmonth_info = clean_mainmonth_info(future_IH, 'IH').sort_index()

# 注意：IH（上证50股指期货）撮合时间是9:30-15：00，CN（富时中国50股指期货）撮合时间是24小时，应该按照IH的时间合并
future_mainmonth_info = pd.merge(future_CN_mainmonth_info, future_IH_mainmonth_info, left_index=True, right_index=True, how='right')
future_mainmonth_info.sort_index(inplace=True)

# 合并汇率信息和主连合约价格信息
future_mainmonth_info = pd.merge(future_mainmonth_info, USDCNY_info, left_index=True, right_index=True, how='left')

In [5]:
# 查看数据缺失程度
future_mainmonth_info.isna().sum(axis=0) / len(future_mainmonth_info)

CN_Bid        0.000000
CN_Ask        0.000000
IH_Bid        0.000000
IH_Ask        0.000064
USDCNY_Bid    0.124666
USDCNY_Ask    0.128043
dtype: float64

In [6]:
# 计算中间价
future_mainmonth_info.fillna(method='ffill')
future_mainmonth_info.dropna(how = 'any',inplace=True)
future_mainmonth_info['CN_mid_price'] = (future_mainmonth_info['CN_Bid'] + future_mainmonth_info['CN_Ask'])/2
future_mainmonth_info['IH_mid_price'] = (future_mainmonth_info['IH_Bid'] + future_mainmonth_info['IH_Ask'])/2
future_mainmonth_info['USDCNY_mid_price'] = (future_mainmonth_info['USDCNY_Bid'] + future_mainmonth_info['USDCNY_Ask'])/2

# 计算经过汇率和合约乘数转换的IH_mid_price
future_mainmonth_info['IH_mid_price_trans'] = 300 * future_mainmonth_info['IH_mid_price'] / future_mainmonth_info['USDCNY_mid_price']

In [7]:
future_mainmonth_info['IH_Bid'] = future_mainmonth_info['IH_Bid'] / future_mainmonth_info['USDCNY_mid_price'] * 300
future_mainmonth_info['IH_Ask'] = future_mainmonth_info['IH_Ask'] / future_mainmonth_info['USDCNY_mid_price'] * 300
future_mainmonth_info = future_mainmonth_info.reset_index()
future_mainmonth_info.rename(columns={'index':'Time',
                                     'CN_Bid':'CN_bid', 'CN_Ask	':'CN_ask', 
                                     'IH_Bid':'IH_bid', 'IH_Ask	':'IH_ask',}, inplace = True)
future_mainmonth_info['CN_bid_qty'] = 100
future_mainmonth_info['CN_ask_qty'] = 100
future_mainmonth_info['IH_bid_qty'] = 100
future_mainmonth_info['IH_ask_qty'] = 100
future_mainmonth_info.drop(['CN_mid_price', 'IH_mid_price','USDCNY_mid_price','IH_mid_price_trans','USDCNY_Bid','USDCNY_Ask'], axis=1, inplace=True)
# save
future_mainmonth_info.to_csv('future_mainmonth_info.csv')

# 平稳性和协整检验
- hurst exponent:
    - https://quantstart.com/articles/Basics-of-Statistical-Mean-Reversion-Testing/
    - https://mp.weixin.qq.com/s?__biz=MzAxMjUyNDQ5OA==&mid=2653580143&idx=1&sn=5085052a337397295bbde63086f184d0&chksm=806e45d2b719ccc49a11f55ac0f150efae231e16301fe74f93ffb6b34f4821925697ded7147d&scene=27

In [ ]:
# seq = future_mainmonth_info['CN_mid_price']
# p_thres = 0.1
# hurst, status, is_stationary, integrated_order = Univariate_Time_Series_Analysis(seq, p_thres).run()

In [ ]:
# seq = future_mainmonth_info['IH_mid_price_trans']
# p_thres = 0.1
# hurst, status, is_stationary, integrated_order = Univariate_Time_Series_Analysis(seq, p_thres).run()

In [ ]:
# seqs = future_mainmonth_info[['CN_mid_price', 'IH_mid_price_trans']]
# p_thres = 0.1
# granger_x_list, is_cointegrated, resid = Multivariate_Time_Series_Analysis(seqs, p_thres).run()
# params, lags, tau = Univariate_Time_Series_Analysis(resid, p_thres).hurst()

In [ ]:
# params

# Strategy Test

In [ ]:
def calculate_spread_zscore(pairs, symbols, lookback=1000):
    # a rolling linear regression between the two closing price time series
    model = RollingOLS(
        endog = pairs[symbols[0]].astype(float),
        exog = sm.add_constant(pairs[symbols[1]]).astype(float),
        window = lookback
    )  # lookahead bias! 
    rres = model.fit()
    params = rres.params.copy()
    
    pairs['hedge_ratio'] = params.iloc[:, 1]
    pairs.dropna(inplace=True)
    
    pairs['spread'] = pairs[symbols[0]] - pairs[symbols[1]] * pairs['hedge_ratio']
    pairs['spread_mean_roll'] = pairs['spread'].rolling(lookback).mean()  # lookahead bias! 
    pairs['spread_std_roll'] = pairs['spread'].rolling(lookback).std()  # lookahead bias! 
    pairs.dropna(inplace=True)
    
    pairs['zscore'] = (pairs['spread'] - pairs['spread_mean_roll']) / pairs['spread_std_roll']
    return pairs

def create_signal(pairs, symbols, z_entry_threshold=2.0, z_exit_threshold=1.0):
    # Calculate when to be long, short and when to exit
    pairs['signal_zscore'] = np.where(pairs['zscore']<=-z_entry_threshold,1,np.NaN)
    pairs['signal_zscore'] = np.where(pairs['zscore']>=z_entry_threshold,-1,pairs['signal_zscore'])
    pairs['signal_zscore'] = np.where(np.abs(pairs['zscore']) <= z_exit_threshold,0,pairs['signal_zscore'])
    pairs['signal_zscore'] = pairs['signal_zscore'].fillna(method = 'ffill')  # lookahead bias! 
    pairs.dropna(inplace=True)
    return pairs

def create_portfolio_returns(pairs, symbols):
    
    last_signal_zscore = 0
    realized_pnl = 0
    realized_pnl_change = False
    hedge_ratio = 0
    side = 0
    backtest_side = 0
    funds_occupation = 0
    entry = False
    
    for t in pairs.index:
        if realized_pnl_change:
            realized_pnl += (pairs.loc[t]['CN_mid_price'] - pairs.loc[t]['IH_mid_price_trans'] * hedge_ratio) * (-side)
            if entry:
                funds_occupation = abs(pairs.loc[t]['CN_mid_price']) + abs(pairs.loc[t]['IH_mid_price_trans'] * hedge_ratio)
        
        if (pairs.loc[t]['signal_zscore'] != last_signal_zscore) & (pairs.loc[t]['signal_zscore']!=0):
            hedge_ratio = pairs.loc[t]['hedge_ratio']
            side = pairs.loc[t]['signal_zscore']
            backtest_side = side
            last_signal_zscore = pairs.loc[t]['signal_zscore']
            realized_pnl_change = True
            entry = True
            
        elif (pairs.loc[t]['signal_zscore'] != last_signal_zscore) & (pairs.loc[t]['signal_zscore']==0):
            hedge_ratio = hedge_ratio
            side = -side
            backtest_side = 0
            last_signal_zscore = pairs.loc[t]['signal_zscore']
            realized_pnl_change = True
            entry = False
            
        else:
            realized_pnl_change = False
            
        pairs.loc[t, 'realized_pnl'] = realized_pnl
        pairs.loc[t, 'backtest_hedge_retio'] = hedge_ratio
        pairs.loc[t, 'backtest_side'] = backtest_side
        pairs.loc[t, 'funds_occupation'] = funds_occupation
        
    pairs['backtest_hedge_retio'] = pairs['backtest_hedge_retio'].shift()
    pairs['backtest_side'] = pairs['backtest_side'].shift()
    pairs = pairs.iloc[1:]
    
    pairs['unrealized_pnl'] = (pairs[symbols[0]] - pairs[symbols[1]]*pairs['backtest_hedge_retio']) * pairs['backtest_side']
    pairs['total_pnl'] = pairs['unrealized_pnl'] + pairs['realized_pnl']
    
    max_funds_occupation = pairs['funds_occupation'].max()
    pairs['accumulated_return'] = (pairs['total_pnl'] + max_funds_occupation)/max_funds_occupation

    return pairs

def evaluate_pnl_pos(portfolio):
    accumulated_return = portfolio['accumulated_return']

    # 进行最大回撤计算
    peak = np.maximum.accumulate(accumulated_return)
    draw_downs = (peak - accumulated_return) / peak
    max_draw_down = np.amax(draw_downs)
    # shape ratio
    shape_ratio = ((accumulated_return.iloc[-1] - 1) - 0.02) / accumulated_return.std()
    # 收益回撤比
    cumret_mdd = (accumulated_return.iloc[-1] - 1) / max_draw_down
    print('最大资金占用：', portfolio['funds_occupation'].max())
    print('期间累计收益：', round((accumulated_return.iloc[-1] - 1) * 100, 2), '%')
    print('期间最大回撤：', round(max_draw_down * 100, 2), '%')
    print('shape ratio：', round(shape_ratio, 2))
    print('收益回撤比：', round(cumret_mdd, 2))
    print('\n')

def plot_pnl_pos(portfolio):
    fig, ax = plt.subplots(4, 1, figsize=(18, 9))
    plt.subplot(411)
    plt.plot(portfolio.index, portfolio['unrealized_pnl'])
    plt.title('未实现收益金额')
    
    plt.subplot(412)
    plt.plot(portfolio.index, portfolio['realized_pnl'])
    plt.title('实现收益金额')
    
    plt.subplot(413)
    plt.plot(portfolio.index, portfolio['total_pnl'])
    plt.title('总收益金额')
    
    plt.subplot(414)
    plt.plot(portfolio.index, portfolio['accumulated_return'])
    plt.title('累计收益率')
    
    plt.tight_layout()  # 调整子图之间的间距以最小化重叠
    plt.show()

In [ ]:
for i in [(i+1)*100 for i in range(17)]:
    print(i)
    # 一手价格，美元计价，考虑了考虑了合约乘数汇率
    pairs_dataframe = future_mainmonth_info[['CN_Bid', 'CN_Ask', 'IH_Bid', 'IH_Ask', 'CN_mid_price', 'IH_mid_price_trans']]
    pairs_dataframe = calculate_spread_zscore(pairs_dataframe, ['CN_mid_price', 'IH_mid_price_trans'], i)
    pairs_dataframe = create_signal(pairs_dataframe, ['CN_mid_price', 'IH_mid_price_trans'])
    pairs_dataframe = create_portfolio_returns(pairs_dataframe, ['CN_mid_price', 'IH_mid_price_trans'])
    evaluate_pnl_pos(pairs_dataframe)
    plot_pnl_pos(pairs_dataframe)

In [ ]:
pairs_dataframe